In [7]:
import asyncio

from gql import Client, gql
from gql.transport.aiohttp import AIOHTTPTransport

pmids = set()


async def get_all_pmids():

    transport = AIOHTTPTransport(url="https://dgidb.org/api/graphql")
    cursor = ""

    async with Client(
        transport=transport,
        fetch_schema_from_transport=True,
    ) as session:
        query = gql(
            """
            query ($cursor:String!) {
              drugs(after:$cursor, last:20) {
                edges {
                  cursor
                  node {
                    interactions {
                      interactionClaims {
                        source {
                          sourceDbName
                        }
                        publications {
                          pmid
                        }
                      }
                    }
                  }
                }
              }
            }            
            """
        )
        while True:
            result = await session.execute(query, variable_values={"cursor": cursor})
            if len(result["drugs"]["edges"]) == 0:
                break
            for edge in result["drugs"]["edges"]:
                cursor = edge["cursor"]
                for interaction in edge["node"]["interactions"]:
                    for interaction_claim in interaction["interactionClaims"]:
                    for pub in interaction["publications"]:
                        pmids.add(str(pub["pmid"]))

await get_all_pmids()

In [8]:
from retraction_audit.audit import RetractionLookup
import os

os.environ["RETRACTION_WATCH_EMAIL"] = "james.sharpsteen@gmail.com"

lookup = RetractionLookup(True)

In [9]:
retractions = []
for pmid in pmids:
    retraction_record = lookup.get_retraction_by_pmid(pmid)
    if retraction_record:
        retractions.append(retraction_record)

In [10]:
len(retractions)

20

In [11]:
retractions

[RetractionData(pmid='11234890', doi='unavailable', article_title='Cotreatment with STI-571 Enhances Tumor Necrosis Factor a-related Apoptosis-inducing Ligand (TRAIL or Apo-2L)â\x80\x93induced Apoptosis of Bcr-Ablâ\x80\x93positive Human Acute Leukemia Cells', journal='Clinical Cancer Research: An Official Journal of the American Association for Cancer Research', authors=['Ramadevi Nimmanapalli', 'Mercedes Porosnicu', 'Diep Nguyen', 'Elizabeth Worthington', 'Erica Oâ\x80\x99Bryan', 'Charles Perkins', 'Kapil N Bhalla'], retraction_watch_refs=[], retraction_doi='10.1158/1078-0432.CCR-19-1602', retraction_pmid='31263034', retraction_reasons=['Duplication of Image', 'Original Data not Provided'], retraction_type=<FlagType.CONCERNED: 'Expression of concern'>),
 RetractionData(pmid='17125593', doi='10.1111/j.1582-4934.2006.tb00533.x', article_title='Differential proteomic profiling to study the mechanism of cardiac pharmacological preconditioning by resveratrol', journal='Journal of Cellular 

In [12]:
from tools import store_results
store_results("DGIDB", len(retractions), retractions, len(pmids))